In [1]:
!pip install langchain

  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchainplus-sdk 0.0.20 requires pydantic<2,>=1, but you have pydantic 2.8.2 which is incompatible.


In [2]:
!pip install -U langchain-community

In [3]:
!pip install pinecone-client

In [4]:
!pip install PyPDF2

In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [6]:
PINECONE_API_KEY = "bce1d75b-83e5-4bfd-80a4-40d878f699b2"
PINECONE_API_ENV = "us-east-1-aws"

In [7]:
import os
file_path = os.path.abspath("data/Medical_book.pdf")
print(file_path)  # This will show the absolute path to your file

# extracted_data = load_pdf(file_path)

/Users/satyambirla/Desktop/Medical-Chatbot/research/data/Medical_book.pdf


In [8]:
!pip install sentence-transformers

In [9]:
#Extract data from the PDF
from langchain.document_loaders import PyPDFLoader

def load_pdf(file_path):
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        return documents
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Now, load the specific PDF
extracted_data = load_pdf("/Users/satyambirla/Desktop/Medical-Chatbot/data/Medical_book.pdf")

In [10]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
!pip install ctransformers

In [12]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

/Users/satyambirla/Desktop/Medical-Chatbot/env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/satyambirla/Desktop/Medical-Chatbot/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
#Initializing the Pinecone
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key="bce1d75b-83e5-4bfd-80a4-40d878f699b2")

# Define the index name
index_name = "medchat"

# Check if the index exists; if not, create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=384, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
#Creating Embeddings for Each of The Text Chunks & storing
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [17]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
import os
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings

# Set the API key directly in the code
os.environ["PINECONE_API_KEY"] = "bce1d75b-83e5-4bfd-80a4-40d878f699b2"

# Initialize the embeddings model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create embeddings for the text chunks
docsearch = Pinecone.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name="medchat"
)

In [19]:
#If we already have an index we can load it like this
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Initialize Pinecone with your API key
pc = Pinecone(api_key="bce1d75b-83e5-4bfd-80a4-40d878f699b2")

# Load the existing index
index = pc.Index(index_name)

# Initialize the Hugging Face model using SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Replace with your model

# Generate the query vector using the model
query = "What are Allergies"
query_vector = model.encode(query)

# Convert the query vector from ndarray to list
query_vector = query_vector.tolist()

# Perform the similarity search
result = index.query(vector=query_vector, top_k=3)

# Process and print the results
for match in result.matches:
    print(f"ID: {match.id}, Score: {match.score}")

ID: ee5dbc3d-4203-444b-badc-cf7a4267f973, Score: 0.682538569
ID: 41a66cf2-ae94-44e3-a039-4d47b085e9ae, Score: 0.682266593
ID: 274d3b70-90f1-48ac-9e24-698f6f557131, Score: 0.678439677


In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm = CTransformers(
    model="/Users/satyambirla/Desktop/Medical-Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt: What are the symptoms of malaria?


/Users/satyambirla/Desktop/Medical-Chatbot/env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Malaria is a serious disease caused by a parasite that is transmitted through the bite of an infected mosquito. The symptoms of malaria can vary depending on the type of parasite and the individual infected, but common symptoms include fever, chills, headache, muscle and joint pain, fatigue, nausea, vomiting, and diarrhea. In severe cases, malaria can cause anemia, seizures, coma, and even death.


Input Prompt: How is HIV transmitted?


Response :  HIV can be transmitted through sexual contact, including vaginal, anal, and oral sex, as well as sharing of needles or other drug paraphernalia. It is also possible for a mother to transmit HIV to her child during pregnancy, childbirth, or breastfeeding.


Input Prompt: How can I treat a common cold at home?


Response :  There are several things you can do at home to help manage symptoms of a common cold:

1. Stay hydrated by drinking plenty of fluids, such as water, tea, or soup.
2. Use over-the-counter medications, such as pain relievers or decongestants, to help alleviate symptoms like fever, headache, or congestion.
3. Rest and relax to help your body recover from the infection.
4. Use saline nasal sprays or drops to help loosen mucus and make breathing easier.
5. Try using a humidifier to add moisture to the air, which can help relieve congestion and cough.

Please let me know if you need anything else!


Input Prompt: What are the common signs of dehydration?


Response :  Dehydration can cause a variety of symptoms including:

* Fatigue or weakness
* Dizziness or lightheadedness
* Headache
* Dry mouth or throat
* Dark urine or decreased urine output
* Constipation or difficulty passing stool
* Muscle cramps or spasms
* Rapid heartbeat or palpitations.
